In [28]:
import os 
from pathlib import Path
import torch
import torchvision.transforms as transforms 
import numpy as np 
import pandas as pd 
from torch.utils.data import DataLoader, Dataset

In [4]:
train_csv_path = os.path.join('/home/anindya/Documents/kaggle/Pawpularity/TRIAL 1 CONCAT/Data/train.csv')
train_df = pd.read_csv(train_csv_path)

features_train_df = np.array(train_df.iloc[:, 1:-1])
transform = transforms.Compose([
    transforms.ToTensor(),
])


In [5]:
f_df = transform(features_train_df)

In [18]:
fdf2 = torch.squeeze(f_df, dim=0)

In [25]:
torch.tensor(features_train_df[1])

tensor([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [46]:
class TabularDataset(Dataset):
    def __init__(self, csv_path, df, train, feature_label_col_dict):
        super(TabularDataset, self).__init__()
        self.csv_path = csv_path
        self.df = df 
        self.train = train 
        self.feature_label_col_dict = feature_label_col_dict
        if df is not None:
            self.data = self.df 
        else:
            self.data = pd.read_csv(self.csv_path)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        feature_start = self.feature_label_col_dict['feature'][0]
        feature_end = self.feature_label_col_dict['feature'][1]
        label = self.feature_label_col_dict['label']

        feature_data = np.array(self.data.iloc[:, feature_start:feature_end])
        label_data = np.array(self.data.iloc[:, label:])
        features = torch.tensor(feature_data[idx]) 
        label = torch.tensor(label_data[idx]).view(1)
        return (features, label)

In [54]:
feature_label_col_dict = {
    'feature' : (1, 13), 
    'label' : -1
}

dataset = TabularDataset(
    csv_path=None, 
    df = train_df, 
    train = None, 
    feature_label_col_dict = feature_label_col_dict
)

In [55]:
loader = DataLoader(dataset, batch_size=64)

In [57]:
dataset[1]

(tensor([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([42]))

In [1]:
from torchvision.models import vgg11
VGG11 = vgg11(pretrained=True)
VGG11 = VGG11.features
for params in VGG11.parameters():
    params.requires_grad = False 

Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /home/anindya/.cache/torch/hub/checkpoints/vgg11-8a719046.pth
100%|██████████| 507M/507M [03:15<00:00, 2.72MB/s]


In [3]:
import torch
x = torch.rand(64, 3, 224, 224)
xp = VGG11(x)
print(xp.shape)

torch.Size([64, 512, 7, 7])


In [6]:
import torch.nn as nn 
fl = nn.Flatten()(x)


In [7]:
fl.shape

torch.Size([64, 150528])